# Process Heterostructure Systems
____

# Import Modules

In [1]:
import sys
import os

import pickle

import numpy as np

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)

from pymatgen.io.ase import AseAtomsAdaptor

from ase.visualize import view
from ase import io

# #############################################################################
# #############################################################################

from methods import process_atom, get_unique_str_repr

# Script Inputs

In [2]:
max_thickness = 4
z_vacuum = 13
frac_constr_supp = 0.6

# Load Data

In [3]:
df = pickle.load(
    open(
        os.path.join(
#             "dataframe_short.pickle",

            os.environ["PROJ_DATA"],
            "01_fe_graph_proj/04_lattice_matching/181218_data",
            "job_dataframe.pickle",
            ),
        "rb"
        )
    )

# Replace NaN with '[]' (empty lists)
for row in df.loc[df.sys_data_i.isnull(), 'sys_data_i'].index:
    df.at[row, 'sys_data_i'] = []

# Unpack heterointerface column into longer dataframe

In [4]:
master_data_list = []
for i_cnt, row_i in df.iterrows():
    dict_i = row_i.to_dict()
    dict_i.pop("sys_data_i")

    for sys_j in row_i["sys_data_i"]:
        dict_j = {**dict_i, **sys_j}
        atoms_i = AseAtomsAdaptor.get_atoms(sys_j["heterointerface"])

        atoms_processed_i = process_atom(
            atoms_i,
            max_thickness=max_thickness,
            z_vacuum=z_vacuum,
            frac_constr_supp=frac_constr_supp,
            )

        dict_j["atoms"] = atoms_processed_i

        master_data_list.append(dict_j)

# Instantiate the dataframe
df_2 = pd.DataFrame(master_data_list)

Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is below support
Graphene is 

# Remove duplicate structures

In [5]:
def get_area(row_i):
    area = np.linalg.norm(np.cross(
        row_i["atoms"].cell[0],
        row_i["atoms"].cell[1],
        ))

    return(area)

df_2["area_2"] = df_2.apply(
    get_area,
    axis=1,
    )


df_2["uniq_str_repr"] = df_2.apply(
    get_unique_str_repr,
    axis=1,
    )

df_3 = df_2.drop_duplicates(subset="uniq_str_repr")

# Removing "long" heterostructures

In [6]:
def structure_square_enough(row_i):
    """
    """
    min_ind = np.argmin([np.linalg.norm(i) for i in row_i["uv1"]])

    ind_list = [0, 1]
    ind_list.remove(min_ind)

    uv_small = row_i["uv1"][min_ind]
    uv_large = row_i["uv1"][ind_list[0]]

    len_u_large = np.linalg.norm(uv_large)
    len_u_small = np.linalg.norm(uv_small)
    large_small_ratio = len_u_large / len_u_small

    if large_small_ratio < 3.:
        return(True)
    else:
        return(False)

In [7]:
df_3["structure_square_enough"] = df_3.apply(
    structure_square_enough,
    axis=1,
    )

df_3 = df_3[df_3["structure_square_enough"]]

/home/raul_desktop/anaconda2/envs/MPInterface/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
# view(df_3[df_3["structure_square_enough"]]["atoms"].tolist())

# np.array(df_3[df_3["structure_square_enough"]]["uniq_str_repr"].tolist())

# Getting rid of structures with too many metal atoms

In [9]:
def num_m_atoms(row_i):
    """
    """
    atoms_i = row_i["atoms"]
#     atoms_i = df_3.iloc[0]["atoms"]

    num_m_atoms = len([i for i in atoms_i.get_atomic_numbers() if i != 6])
#     num_c_atoms = len([i for i in atoms_i.get_atomic_numbers() if i == 6])

#     out_dict = {
#         "metal": num_m_atoms,
#         "carbon": num_c_atoms,
#         }

    return(num_m_atoms)

In [10]:
df_3["num_m_atoms"] = df_3.apply(
    num_m_atoms,
    axis=1,
    )

df_3 = df_3[df_3["num_m_atoms"] <= 60]

In [11]:
print(len(df_3))

74


In [12]:
# df_3

In [40]:
view(df_3["atoms"].tolist())

In [39]:
pickle.dump(
df_3,
open(
"processed_dataframe.pickle",
"wb"
)
)

In [14]:
# def num_atoms(row_i):
#     """
#     """
#     atoms_i = row_i["atoms"]
# #     atoms_i = df_3.iloc[0]["atoms"]

#     num_m_atoms = len([i for i in atoms_i.get_atomic_numbers() if i != 6])
#     num_c_atoms = len([i for i in atoms_i.get_atomic_numbers() if i == 6])

#     out_dict = {
#         "metal": num_m_atoms,
#         "carbon": num_c_atoms,
#         }

#     return(out_dict)

In [15]:
# df_3["num_atoms"] = df_3.apply(
#     num_atoms,
#     axis=1,
#     )

In [16]:
# print(num_m_atoms)
# print(num_c_atoms)

In [17]:
# view(atoms_i)

In [18]:
# view(df_2["atoms"].tolist())

# len(df_2)

In [19]:
# # view(df_2.iloc[0]["atoms"])

# atoms = df_2.iloc[0]["atoms"]

# if atoms.cell[1][1] < 0.:
#     atoms.cell[1] = -1 * atoms.cell[1]

#     atoms.set_cell(
#         atoms.cell,
#         scale_atoms=True,
#         )

# atoms.cell[1] = -1 * atoms.cell[1]

# atoms.set_cell(
#     atoms.cell,
#     scale_atoms=True,
#     )

In [20]:
# for i_cnt, row_i in df_3.iterrows():
#     tmp = 42

# row_i["atoms"].cell[0]

# x = [-11.097, -6.4, 0]
# y = [8.631, -14.95, 0]
# np.linalg.norm(np.cross(x, y))

In [21]:
# for i_cnt, row_i in df_3.iterrows():
# #     tmp = 42

#     tmp = structure_square_enough(row_i)
#     print(tmp)

In [22]:
# len(df)
# df.info(memory_usage="deep")

# df_3.info(memory_usage="deep")

In [23]:
# tmp1 = df_2["uniq_str_repr"].tolist()
# print(len(tmp1))


# len(list(set(tmp1)))

In [24]:
# len(df_2.drop_duplicates(subset="uniq_str_repr"))

In [25]:
# print(df_2.iloc[0]["atoms"].cell)
# print("")
# print(df_2.iloc[1]["atoms"].cell)

In [26]:
# def get_unique_str_repr(row_i):
#     """
#     """
#     dupl_prop_dict = {
#     #     "": row_i[],
#     #     "facet": str(row_i["facet"]),  # Not facet, some may be equiv.

#         "crystal": row_i["crystal"],
#         "metal": row_i["metal"],
#         "min_area": '{:.5e}'.format(row_i["min_area"]),
#         "u_mismatch": '{:.5e}'.format(row_i["u_mismatch"]),
#         "v_mismatch": '{:.5e}'.format(row_i["v_mismatch"]),
#         "angle_mismatch": '{:.5e}'.format(row_i["angle_mismatch"]),
#         "cell_x": str(['{:.5e}'.format(i) for i in row_i["atoms"].cell[0]]),
#         "cell_y": str(['{:.5e}'.format(i) for i in row_i["atoms"].cell[1]]),
#         }

#     str_repr_i = ""
#     for key, value in dupl_prop_dict.items():
#         str_repr_i += value + " | "

#     return(str_repr_i)

In [27]:
# atoms_processed_i = process_atom(
#     atoms,
#     max_thickness=max_thickness,
#     z_vacuum=z_vacuum,
#     frac_constr_supp=frac_constr_supp,
#     )

In [28]:
# def get_sum_force(row):
#     """
#     """
#     #| - get_sum_force
#     if row["atoms_object"] is not None:
#         try:
#             max_force_tmp = max_force(row["atoms_object"][-1])
#             return(max_force_tmp[1])

#         except:
#             return(None)

#     else:
#         return(None)
#     #__|

# df_master["sum_force"] = df_master.apply(
#     get_sum_force,
#     axis=1,
#     )

In [29]:
# dict_i = row_i.to_dict()
# dict_i.pop("sys_data_i")

In [30]:
# io.write?
# # atoms

# io.write("temp.traj", atoms)

In [31]:
# dict1 = {
#     "a": 1,
#     "b": 2,
#     "c": 3,
#     }

In [32]:
# for i_cnt, row_i in df.iterrows():
#     tmp = 42

# for sys_j in row_i["sys_data_i"]:
#     tmp = 42

# atoms = AseAtomsAdaptor.get_atoms(sys_j["heterointerface"])

In [33]:
# /mnt/c/Users/raul_desktop/Dropbox/01_norskov/04_comp_clusters/02_DATA
# /01_fe_graph_proj/04_lattice_matching/181218_data

In [34]:
# df_short = df.sample(n=2)
# pickle.dump(
#     df_short,
#     open(
#         "dataframe_short.pickle",
#         "wb"
#         )
#     )

In [35]:
# sys.path.insert(
#     1,
#     os.path.join(

#         "/home/raul_desktop/anaconda2",
#         "envs/MPInterface/lib",
#         "python3.6/site-packages/pymatgen",
# #         os.environ["PROJ_DATA"],
# #         "01_fe_graph_proj/04_lattice_matching/181218_data",
#         )
#     )